In [27]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import f1_score
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [28]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]

In [29]:
dset_idx = 2

In [30]:
seed = 123
rand = np.random.RandomState()

In [31]:
condition_map = {"healthy": 0, "tumor": 1}

# Load normalized matrisome count data

In [32]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/norm_matrisome_counts.tsv", sep='\t')
coldata_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/coldata.tsv", sep="\t")
counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df, "geneID", "sample_name"
)
print(counts_t_df.shape)

(246, 1009)


In [33]:
joined_df = (
    pd.merge(coldata_df, counts_t_df, on="sample_name")
        .set_index("sample_name")
        .drop("data_source", axis=1)
        .assign(condition = lambda df: df.condition.apply(lambda x: condition_map[x]))
)

In [34]:
joined_df.condition.value_counts()

1    141
0    105
Name: condition, dtype: int64

# Prep data

In [35]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

## Get baselines

In [36]:
rand.seed(seed)

label_value_counts_df = (
    pd.DataFrame(y_df.condition.value_counts()).reset_index()
        .rename(columns={"index": "label", "condition": "n"})
        .sort_values("n", ascending=False)
)

most_frequent_label = label_value_counts_df.label[0]
most_frequent_baseline = f1_score(y_df.values.squeeze(), np.repeat(most_frequent_label, y_df.shape[0]), average="macro")

mc_baseline = opt.mc_classification_baseline(
    y=y_df.values.squeeze(),
    labels=label_value_counts_df.label.values,
    weights=label_value_counts_df.n.values / label_value_counts_df.n.values.sum(),
    metric=lambda y, yhat: f1_score(y, yhat, average="macro"),
    n=1001
)

print(f"Most frequent baseline: {most_frequent_baseline}")
print(f"Monte Carlo baseline: {mc_baseline.mean()}")

Most frequent baseline: 0.3643410852713178
Monte Carlo baseline: 0.5004461112489567


## SMBO

In [37]:
matrisome_genes = counts_t_df.columns[1:]

def objective(h_params, X, y, penalty_default, scoring_default, r, verbose=True):
    if verbose:
        print(h_params)

    model = make_pipeline(
        ColumnTransformer([
            ("standard", StandardScaler(), list(matrisome_genes))
        ], remainder="passthrough"),
        LogisticRegression(
            C=h_params[0],
            l1_ratio=h_params[1],
#             solver=h_params[1],
            solver="saga",
            penalty=penalty_default,
            n_jobs=-1,
            random_state=r
        )
    )
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring=scoring_default
    ))

In [38]:
l1_space = [
    Real(1e-1, 1e1, name="C"),
    Categorical(["liblinear", "saga"], name="solver")
]

elasticnet_space = [
    Real(1e-1, 1e1, name="C"),
    Real(0, 1, name="l1_ratio")
]

n_initial = 10 * (len(elasticnet_space) + 1)
n_calls = 50 * (len(elasticnet_space) + 1)

In [39]:
penalty_default = "elasticnet"
scoring_default = "f1_macro"
space = elasticnet_space
callback_file = f"{unified_dsets[dset_idx]}_opt_lr_cancer_y_n_h_params_{penalty_default}_{scoring_default}.tsv"

try:
    os.remove(callback_file)
except OSError:
    pass

res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, penalty_default, scoring_default, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls=n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n=5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[7.15825767317995, 0.4284709261868258]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.3600
Function value obtained: -0.9781
Current minimum: -0.9781
Iteration No: 2 started. Evaluating function at random point.
[6.939760064765932, 0.7191503101547732]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.0834
Function value obtained: -0.9781
Current minimum: -0.9781
Iteration No: 3 started. Evaluating function at random point.
[4.962077440982714, 0.7800277619120793]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.9727
Function value obtained: -0.9781
Current minimum: -0.9781
Iteration No: 4 started. Evaluating function at random point.
[4.1681512898192885, 0.5796942970226102]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.9155
Function value obtained: -0.9835
Current minimum: -0.9835
Iteration No: 5 started. Evaluating function at random point

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 95 ended. Search finished for the next optimal point.
Time taken: 1.3847
Function value obtained: -0.9863
Current minimum: -0.9876
Iteration No: 96 started. Searching for the next optimal point.
[1.5195249584878407, 0.00037122752799578784]
Iteration No: 96 ended. Search finished for the next optimal point.
Time taken: 1.4934
Function value obtained: -0.9822
Current minimum: -0.9876
Iteration No: 97 started. Searching for the next optimal point.
[9.46643566299312, 0.009024122171422679]


KeyboardInterrupt: 